# Cleanning notebook 

Let's first call the libraries I'll use : 

In [1]:
data_path = "~/Documents/openclassroom/data/"

# On importe les librairies dont on aura besoin pour ce tp
import numpy as np
import pandas as pd
## plot : 
import matplotlib
import matplotlib.pyplot as plt
matplotlib.rcParams['figure.figsize'] = [15, 5]
## stat / model : 
from sklearn.model_selection import train_test_split

import itertools as IT

# 1. Import data : <!-- I have decided to extract the data labeled with the french nutriscore, to set a cleanning method and then decide the application I want to propose.  -->

In [21]:
filename = data_path+"en.openfoodfacts.org.products.csv"
# filename = data_path+"openfoodfacts_with_cholesterol.csv"
# df = pd.read_csv(filename,"\t")

## GET INDEX OF NUTRI SCORE IN COLUMNS : 
tmp = pd.read_csv(filename, sep = "\t").iloc[0,:].index
col_index_nutri_score_fr = np.where(tmp.index=="nutrition-score-fr_100g")[0][0]

In [59]:
def valid(chunks):
    for chunk in chunks:
        mask = ~chunk.iloc[:,p].isna().values
        if mask.all():
            yield chunk
        else:
            yield chunk.loc[mask]
            break

chunksize = 10 ** 4
chunks = pd.read_csv(filename, sep = "\t", low_memory=False, chunksize=chunksize, header=None)
df_original = pd.concat(valid(chunks))
df_original.head()

In [55]:
p = 175
~chunk.iloc[:,p].isna()

-5465

In [48]:
tmp = pd.read_csv(filename, sep = "\t").iloc[0,:].index
p = col_index_nutri_score_fr = np.where(tmp.index=="nutrition-score-fr_100g")[0][0]

175

In [25]:
# df_original = (pd.read_csv(filename,"\t") [lambda x : np.any([~x["nutrition-score-fr_100g"].isna().values, 
#                                                               x["countries"]=="France"], axis = 0)])
df_original = (pd.read_csv(filename,"\t", low_memory=False) [lambda x : ~x["nutrition-score-fr_100g"].isna()])
df_original.shape

(152, 172)

In [21]:
dtype_characteristics = {"code":"category","url":str,"creator":str,"created_t":str,
                           "created_datetime":str,"last_modified_t":str,
                           "last_modified_datetime":str,"product_name":str,
                           "generic_name":str,"quantity":int}

In [ ]:
df = df_original.sample(10000, replace = False)
n = df.shape[0]
# del df_original 
df.head(3)

In [15]:
# nutrition-score-fr_100g
sum(df_original["nutrition-score-fr_100g"].isna().values)

0

In [9]:
np.array(pd.read_csv(filename,"\t").countries)

/home/clairegayral/.env_python/openclassroom/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,8,13,19,20,21,22,23,27,28,29,31,32,38,40,41,42,47,52,55,64,79,82) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


array(['France', 'France', 'en:es', ..., 'en:be', 'en:france', 'France'],
      dtype=object)

In [32]:
# filename = data_path+"openfoodfacts_with_cholesterol.csv"
# df_original = (pd.read_csv(filename,"\t") [lambda x :  x["nutrition-score-fr_100g"].isna().values])
#                  dtype={"code":"category","url":str,"creator":"category",
#                         "created_t"None, "last_modified_t":None})

The data is huge (1 716 277 products).
Let's first work on a sample of size 10 000, and then comment this part : 

In [11]:
x = df

# [~x["nutrition-score-fr_100g"].isna().values, ~x["nutrition-score-en_100g"].isna().values]
np.any([~x["nutrition-score-fr_100g"].isna().values, ~x["inositol_100g"].isna().values], axis = 0)

NameError: name 'df' is not defined

In [ ]:
df.columns

## 1.1 Drop Missing values : 
Let's remove the variables with too much NaN. First, I remove the columns filled with NaNs (there is no information in them). 

Then, I have chosen to keep the 1st quantile value of the number of NaNs in each variables : 
* if the variable has less NaNs than this quantile, I will estimate the missing value with KNN procedure, 
* else, I drop the corresponding column. 

In [36]:
df = df.copy()
## DROP COL FILLED WITH NANs
df = df.loc[:,df.isna().sum(axis=0)<n]
print("There are ", sum(df.isna().sum(axis=0)==n), "variables without any information (all NaNs)")

## DROP COL WITH MORE NANs THAN THE 1st QUANTILE :
nan_repartition = df.isna().sum(axis=0)
# nan_threshold = nan_repartition.quantile(0.25)
nan_threshold = nan_repartition.mean()
print("I drop variables with more than", nan_threshold, "NaNs")
df = df.drop(df.columns[nan_repartition>nan_threshold], axis = 1)
df.describe()

There are  0 variables without any information (all NaNs)
I drop variables with more than 0.0 NaNs


,created_t,last_modified_t
count,1.000000e+04,1.000000e+04
mean,1.550166e+09,1.585051e+09
std,4.922722e+07,2.868432e+07
min,1.332618e+09,1.353474e+09
25%,1.518806e+09,1.576339e+09
50%,1.561571e+09,1.587659e+09
75%,1.587662e+09,1.606334e+09
max,1.617654e+09,1.617655e+09


## 1.2 Exploration of the different values :

Then, I used the description data in https://world.openfoodfacts.org/data/data-fields.txt, to separate the global data frame into sub-frames. 
Note : to transform the .txt into these lists of variables, I used the bash command : 

    cat data-fields.txt | cut -d " " -f 1 | sed s/://g | sed s/\$/\",/g | sed s/\^/\"/g > data_fields2.txt

In [35]:
# USE BASH cat data-fields.txt | cut -d " " -f 1 | sed s/://g | sed s/\$/\",/g | sed s/\^/\"/g > data_fields2.txt
list_of_characteristics = ["code","url","creator","created_t","created_datetime","last_modified_t",
                           "last_modified_datetime","product_name","generic_name","quantity"]
list_of_tags = ["packaging", "packaging_tags", "brands", "brands_tags", "categories",
                "categories_tags", "categories_fr","origins", "origins_tags","manufacturing_places",
                "manufacturing_places_tags", "labels", "labels_tags", "labels_fr", "emb_codes", 
                "emb_codes_tags", "first_packaging_code_geo", "cities", "cities_tags", 
                "purchase_places", "stores", "countries", "countries_tags", "countries_fr"]
list_of_ingredients = ["ingredients_text", "traces", "traces_tags"]
list_of_misc = ["serving_size", "no_nutriments", "additives_n", "additives", "additives_tags", 
                "ingredients_from_palm_oil_n", "ingredients_from_palm_oil", 
                "ingredients_from_palm_oil_tags", "ingredients_that_may_be_from_palm_oil_n", 
                "ingredients_that_may_be_from_palm_oil", "ingredients_that_may_be_from_palm_oil_tags", 
                "nutrition_grade_fr", "main_category", "main_category_fr", "image_url", 
                "image_small_url"]
list_of_nutri_facts = ["energy_100g","energy-kj_100g","energy-kcal_100g","proteins_100g","casein_100g",
                       "serum-proteins_100g","nucleotides_100g","carbohydrates_100g","sugars_100g",
                       "sucrose_100g","glucose_100g","fructose_100g","lactose_100g","maltose_100g",
                       "maltodextrins_100g","starch_100g","polyols_100g","fat_100g","saturated-fat_100g",
                       "butyric-acid_100g","caproic-acid_100g","caprylic-acid_100g","capric-acid_100g",
                       "lauric-acid_100g","myristic-acid_100g","palmitic-acid_100g","stearic-acid_100g",
                       "arachidic-acid_100g","behenic-acid_100g","lignoceric-acid_100g","cerotic-acid_100g",
                       "montanic-acid_100g","melissic-acid_100g","monounsaturated-fat_100g",
                       "polyunsaturated-fat_100g","omega-3-fat_100g","alpha-linolenic-acid_100g",
                       "eicosapentaenoic-acid_100g","docosahexaenoic-acid_100g","omega-6-fat_100g",
                       "linoleic-acid_100g","arachidonic-acid_100g","gamma-linolenic-acid_100g",
                       "dihomo-gamma-linolenic-acid_100g","omega-9-fat_100g","oleic-acid_100g",
                       "elaidic-acid_100g","gondoic-acid_100g","mead-acid_100g","erucic-acid_100g",
                       "nervonic-acid_100g","trans-fat_100g","cholesterol_100g","fiber_100g","sodium_100g",
                       "alcohol_100g","vitamin-a_100g","vitamin-d_100g","vitamin-e_100g","vitamin-k_100g",
                       "vitamin-c_100g","vitamin-b1_100g","vitamin-b2_100g","vitamin-pp_100g",
                       "vitamin-b6_100g","vitamin-b9_100g","vitamin-b12_100g","biotin_100g",
                       "pantothenic-acid_100g","silica_100g","bicarbonate_100g","potassium_100g",
                       "chloride_100g","calcium_100g","phosphorus_100g","iron_100g","magnesium_100g",
                       "zinc_100g","copper_100g","manganese_100g","fluoride_100g","selenium_100g",
                       "chromium_100g","molybdenum_100g","iodine_100g","caffeine_100g","taurine_100g",
                       "ph_100g","fruits-vegetables-nuts_100g","carbon-footprint_100g",
                       "nutrition-score-fr_100g","nutrition-score-uk_100g"]

# df_caracteristics = df[df.columns.intersection(list_of_characteristics)]
# df_tags = df[df.columns.intersection(list_of_tags)]
# df_ingredients = df[df.columns.intersection(list_of_ingredients)]
# df_misc = df[df.columns.intersection(list_of_misc)]
df_nutri = df[df.columns.intersection(list_of_nutri_facts)]
df_nutri.describe()

ValueError: Cannot describe a DataFrame without columns

In [19]:
for var in list_of_tags[:4] : 
    print("\n",var)
    if var in df.columns:
        tmp = df[df.columns.intersection([var])]
        var_values = tmp.iloc[(~pd.isna(tmp).values)]
        if len(var_values) < 8:
            print("values = ", var_values.values)
        else :
            print("ex of values = ", np.array(tmp.iloc[(~pd.isna(tmp).values)].sample(n=8).values).transpose()[0])
    else : 
        print("no data")


 packaging
no data

 packaging_tags
no data

 brands
ex of values =  ['Yoplait' 'Santiveri' 'Buenerkemper & Howe Company'
 'Monoprix Gourmet,Monoprix' 'Dolciando' 'Franprix Bio,Franprix'
 'simple truth, Simple Truth Organic, The Kroger Co.' 'Alteza']

 brands_tags
ex of values =  ['roundy-s' 'man-fong' 'favorina' 'leclerc' 'poppies'
 '365,365-everyday-value' 'brinkers' 'pascual']


In [21]:
df.columns

Index(['code', 'url', 'creator', 'created_t', 'created_datetime',
       'last_modified_t', 'last_modified_datetime', 'product_name', 'brands',
       'brands_tags', 'categories', 'categories_tags', 'categories_en',
       'countries', 'countries_tags', 'countries_en', 'pnns_groups_1',
       'pnns_groups_2', 'states', 'states_tags', 'states_en', 'main_category',
       'main_category_en', 'image_url', 'image_small_url',
       'image_nutrition_url', 'image_nutrition_small_url', 'energy-kcal_100g',
       'energy_100g', 'fat_100g', 'saturated-fat_100g', 'carbohydrates_100g',
       'sugars_100g', 'proteins_100g', 'salt_100g', 'sodium_100g'],
      dtype='object')

# 1.3 Estimate remaining missing values